In [1]:
from llm.context.llm_context import LLMContext
from llm.strategy.big_llama_model import BigLlamaModel
from llm.strategy.small_llama_model import SmallLlamaModel
from llm.strategy.big_mistral_model import BigMistralModel

In [2]:
data = {}
data["system"] = "You are a helpful AI assistant for travel tips and recommendations"
data["user"] = "What can you help me with?"

In [3]:
context = LLMContext(BigMistralModel())
context.generate_response(data)

' I can help you with a variety of travel-related tasks, such as:\n\n* Researching and recommending destinations based on your interests and preferences\n* Providing information on visa requirements, weather, and cultural customs for different countries\n* Suggesting activities and attractions to visit in a specific location\n* Recommending accommodation options in different price ranges\n* Providing tips on how to save money while traveling\n* Help you to plan your itinerary\n* Recommending local restaurants and cuisines\n* Providing information on transportation options, including flights, trains, and car rentals\n* And many more, I am here to help you to have a smooth and enjoyable trip. Please let me know what specifically you need help with.'

In [4]:
context = LLMContext(BigLlamaModel())
context.generate_response(data)

" I'd be delighted to assist you with all your travel-related queries! Here are some ways I can help:\n\n1. **Destination Recommendations**: Looking for a perfect getaway? I can suggest destinations based on your interests, budget, and preferences.\n\n2. **Itinerary Planning**: Need help planning your trip? I can assist you in creating a personalized itinerary, including activities, accommodations, and transportation.\n\n3. **Travel Tips and Advice**: Want to know the best ways to navigate a new city, or how to pack efficiently? I've got you covered with expert travel tips and advice.\n\n4. **Accommodation and Activity Bookings**: I can provide recommendations and help you book accommodations, flights, and activities, making your travel planning a breeze.\n\n5. **Cultural Insights and Local Secrets**: Want to experience a destination like a local? I can share cultural insights, hidden gems, and off-the-beaten-path recommendations.\n\n6. **Travel Safety and Precautions**: Concerned abou

In [5]:
context = LLMContext(SmallLlamaModel())
context.generate_response(data)

" I'd be happy to help you with your travel plans! I can assist you with a wide range of topics, including:\n\n1. Destination recommendations: I can suggest popular and off-the-beaten-path destinations, based on your interests and preferences.\n2. Itinerary planning: I can help you plan your trip, including creating a daily schedule, recommending activities, and booking accommodations.\n3. Travel tips and advice: I can provide you with insider knowledge on how to navigate airports, public transportation, and cultural differences.\n4. Packing lists: I can help you create a packing list tailored to your destination and activities.\n5. Language translation: I can translate phrases and sentences for you, helping you communicate with locals during your trip.\n6. Budgeting and cost-saving tips: I can provide you with budget-friendly options for accommodations, food, and activities.\n7. Cultural insights: I can share information about local customs, traditions, and etiquette to help you bette